<div align="center">
 **Intel Image Classification Dataset **
</div>


In [ ]:
import numpy as np
import pandas as pd
import keras
from keras.datasets import mnist
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from PIL import Image
import matplotlib.pyplot as plt
import kagglehub
import os

In [ ]:
path = kagglehub.dataset_download("puneet6060/intel-image-classification")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'intel-image-classification' dataset.
Path to dataset files: /kaggle/input/intel-image-classification


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
class CNN(nn.Module):
    def __init__(self, in_channels = 3, num_classes = 7):
        super(CNN, self).__init__()

        self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 16, kernel_size = (3,3), stride = (1,1), padding = (1,1))
        self.pool = nn.MaxPool2d(kernel_size = (2,2), stride = (2,2))
        self.conv2 = nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size = (3,3), stride = (1,1), padding = (1,1))
        self.conv3 = nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = (3,3), stride = (1,1), padding = (1,1))
        self.dropout = nn.Dropout(0.25)
        self.fc1 = nn.Linear(64*8*8, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.conv3(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.dropout(x)
        x = self.fc1(x)

        return x

In [ ]:
# Hyperparameters
in_channels = 3
num_classes = 6
learning_rate = 0.001
batch_size = 128
num_epochs = 100

In [ ]:
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5,0.5,0.5], std=[0.5,0.5,0.5])
])


In [ ]:
train_dataset = datasets.ImageFolder(
    root=os.path.join(path,'seg_train','seg_train'),
    transform=transform
)

test_dataset = datasets.ImageFolder(
    root=os.path.join(path,'seg_test','seg_test'),
    transform=transform
)


In [ ]:
train_data = DataLoader(
    train_dataset,
    batch_size = batch_size,
    shuffle=True,
    num_workers=4
)

test_data = DataLoader(
    test_dataset,
    batch_size = batch_size,
    num_workers=4
)
"""
print("Classes:", train_dataset.classes)
print("Number of training images:", len(train_dataset))
print("Number of validation images:", len(test_dataset))

# Optional: check a batch
images, labels = next(iter(train_data))
print("Batch image shape:", images.shape)
print("Batch label shape:", labels.shape)
"""

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


'\nprint("Classes:", train_dataset.classes)\nprint("Number of training images:", len(train_dataset))\nprint("Number of validation images:", len(test_dataset))\n\n# Optional: check a batch\nimages, labels = next(iter(train_data))\nprint("Batch image shape:", images.shape)\nprint("Batch label shape:", labels.shape)\n'

In [ ]:
model = CNN()
loss_criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)
model.to(device)

CNN(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (dropout): Dropout(p=0.25, inplace=False)
  (fc1): Linear(in_features=4096, out_features=7, bias=True)
)

In [ ]:
# Training

i = 0
for epoch in range(num_epochs):
    i += 1
    model.train()
    running_loss = 0.0

    for images, labels in train_data:
        images, labels = images.to(device), labels.to(device)
        labels = labels.long()
        optimizer.zero_grad()
        output = model(images)
        loss = loss_criterion(output, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    epoch_loss = running_loss/len(train_data)
    print(i, " Epoch: ", (epoch+1)/(num_epochs), " Loss: ", epoch_loss)


1  Epoch:  0.01  Loss:  1.1281343606385317
2  Epoch:  0.02  Loss:  0.8691366228190336
3  Epoch:  0.03  Loss:  0.7823724378239024
4  Epoch:  0.04  Loss:  0.7050882529128681
5  Epoch:  0.05  Loss:  0.6668708503246308
6  Epoch:  0.06  Loss:  0.6248153513128107
7  Epoch:  0.07  Loss:  0.5921637759967284
8  Epoch:  0.08  Loss:  0.5600108954039487
9  Epoch:  0.09  Loss:  0.5344286693768068
10  Epoch:  0.1  Loss:  0.520687382329594
11  Epoch:  0.11  Loss:  0.5051428989930586
12  Epoch:  0.12  Loss:  0.4786868201060729
13  Epoch:  0.13  Loss:  0.45537140206857163
14  Epoch:  0.14  Loss:  0.45268621038306844
15  Epoch:  0.15  Loss:  0.42970964312553406
16  Epoch:  0.16  Loss:  0.41745166968215597
17  Epoch:  0.17  Loss:  0.39994765398177234
18  Epoch:  0.18  Loss:  0.38517884747548536
19  Epoch:  0.19  Loss:  0.3704803767529401
20  Epoch:  0.2  Loss:  0.3635573588988998
21  Epoch:  0.21  Loss:  0.33926655026999386
22  Epoch:  0.22  Loss:  0.3443336684595455
23  Epoch:  0.23  Loss:  0.3309896137

In [ ]:
train_dataset.class_to_idx

{'buildings': 0,
 'forest': 1,
 'glacier': 2,
 'mountain': 3,
 'sea': 4,
 'street': 5}

In [ ]:
# Testing
model.eval()
correct = 0
total = 0
for images, labels in test_data:
  images, labels = images.to(device), labels.to(device)
  images = images.float()
  labels = labels.long()
  output = model(images)
  loss = loss_criterion(output, labels)
  correct += (output.argmax(1) == labels).sum().item()
  total += labels.size(0)
print(100*correct/total)

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


81.33333333333333


In [ ]:
pred_image = Image.open("/content/drive/MyDrive/Colab Notebooks/WiDS Assignments/2/seg_pred/seg_pred/76.jpg")
image = Image.open("/content/drive/MyDrive/forest.png")

transformer = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.Lambda(lambda image: image.convert("RGB")),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5,0.5,0.5], std=[0.5,0.5,0.5])
])
image = transformer(image)
image.to(device)
print(image.shape)


torch.Size([3, 64, 64])


In [ ]:
model.eval()
image = image.unsqueeze(0)
image = image.to(device)
with torch.no_grad():
    output = model(image)
    prediction = output.argmax(dim=1)

my_dict = train_dataset.class_to_idx
target_value = prediction.item()

found_key = next((key for key, val in my_dict.items() if val == target_value), None)

print("Predicted class:", found_key)

probs = F.softmax(output, dim=1)

print(probs)

Predicted class: forest
tensor([[3.1973e-14, 1.0000e+00, 4.0406e-09, 1.1863e-15, 1.6100e-18, 1.0752e-18,
         1.8981e-24]], device='cuda:0')
